In [ ]:
from llama_index.llms.bedrock import Bedrock

llm = Bedrock(model="anthropic.claude-3-sonnet-20240229-v1:0")     

#response = llm.complete("Paul Graham is ")

In [ ]:
import json
from llama_index.embeddings.bedrock import BedrockEmbedding

embed_model = BedrockEmbedding()

#supported_models = BedrockEmbedding.list_supported_models()

#print(json.dumps(supported_models, indent=2))

In [ ]:
from llama_index.readers.file import UnstructuredReader

loader = UnstructuredReader()

filename = "Sach-EA-2023-04-06-Police-Version Nr 23(FLExA).pdf"

documents = loader.load_data(f"data/{filename}")
documents[0].text
documents[0].to_dict()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=1000)

nodes = parser.get_nodes_from_documents(documents)
nodes[0].text

nodes[:5]

In [ ]:
from llama_index.core import Document

documents = []
meta = {}
for element in elements:
    metadata = element.metadata.to_dict()
    meta['page_label'] = metadata.get('page_number', None)
    meta["file_name"] = metadata["filename"]
    meta["file_path"] = metadata["file_directory"]
    meta["file_type"] = metadata["filetype"]
    documents.append(Document(text=element.text, metadata=meta))

documents[:5]

In [ ]:
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

Settings.llm = Bedrock(model="anthropic.claude-v1")
Settings.embed_model = embed_model
Settings.chunk_size = 512

#reader = SimpleDirectoryReader(input_files=["./data/Agile_Grundlagen_2024_handout_3.pdf"])

# only load pdf files
required_exts = [".pdf"]

reader = SimpleDirectoryReader(input_dir="./data",
    required_exts=required_exts,
    recursive=True
    )

documents = reader.load_data()
print(f"Loaded {len(documents)} pages.")

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=1000)

nodes = parser.get_nodes_from_documents(documents)
nodes[0].text

In [ ]:
nodes[:5]

In [ ]:
import chromadb
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore

chroma_client = chromadb.EphemeralClient()

chroma_collection = chroma_client.create_collection("midcorp")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

rag_application = index.as_query_engine()

In [ ]:
from unstructured.partition.pdf import partition_pdf

elements = partition_pdf("./data/Agile_Grundlagen_2024_handout_3.pdf", strategy='hi_res', include_page_breaks=True, infer_table_structure=False, languages=['deu'])    


In [ ]:
# from unstructured.chunking.title import chunk_by_title

# chunks = chunk_by_title(elements, max_characters=1000)

# for z, chunk in enumerate(chunks):
#     print(f"Chunk {z}:\n", chunk)

In [ ]:
from llama_index.core import Document

doc = Document(text="This is a test document.")
doc

In [ ]:
from llama_index.core import Document

documents = []
meta = {}
for element in elements:
    metadata = element.metadata.to_dict()
    meta['page_label'] = metadata.get('page_number', None)
    meta["file_name"] = metadata["filename"]
    meta["file_path"] = metadata["file_directory"]
    meta["file_type"] = metadata["filetype"]
    documents.append(Document(text=element.text, metadata=meta))

documents[:5]

In [ ]:
import chromadb
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore

chroma_client = chromadb.EphemeralClient()

chroma_collection = chroma_client.create_collection("Alex")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

documents = SimpleDirectoryReader(input_dir="./data",
    required_exts=required_exts,
    recursive=True
    ).load_data()

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

rag_application = index.as_query_engine()

In [ ]:
index = VectorStoreIndex.from_documents(
    documents=documents,
    embed_model=embed_model
)

rag_application = index.as_query_engine()

In [ ]:
from deepeval.integrations.llama_index import DeepEvalFaithfulnessEvaluator

# An example input to your RAG application
user_input = "What is this?"

# LlamaIndex returns a response object that contains
# both the output string and retrieved nodes
#response_object = rag_application.query(user_input)


In [ ]:
evaluator = DeepEvalFaithfulnessEvaluator()

evaluation_result = evaluator.evaluate_response(
    query=user_input, response=response_object
)
print(evaluation_result)

In [ ]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [ ]:
index = VectorStoreIndex.from_documents(
    documents=documents,
    embed_model=embed_model
)

rag_application = index.as_query_engine()

In [ ]:
retriever = index.as_retriever(similarity_top_k=6)

In [ ]:
question = "Was ist die Definition von agiler Arbeitsweise?"

contexts = retriever.retrieve(question)
contexts

In [ ]:
context_list = [n.get_content() for n in contexts]
len(context_list)

In [ ]:
context_list

In [ ]:
from llama_index.core.query_engine import MultiStepQueryEngine
from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform

step_decompose_transform = StepDecomposeQueryTransform(llm=llm, verbose=True)

In [ ]:
query_engine = index.as_query_engine(llm=llm)

query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    num_steps=3,
    query_transform=step_decompose_transform,
    index_summary="Used to answer questions about agile methodology."
)
response_gpt4 = query_engine.query(str_or_query_bundle = "Was ist die Definition von agiler Arbeitsweise?")

In [ ]:
from llama_index.core import PromptTemplate

query_gen_str = """\
You are a helpful assistant that generates multiple search queries based on a \
single input query. Generate {num_queries} search queries, one on each line, \
related to the following input query:
Query: {query}
Queries:
"""
query_gen_prompt = PromptTemplate(query_gen_str)

llm = Bedrock(model="amazon.titan-text-express-v1")      # anthropic.claude-v2

def generate_queries(query: str, llm, num_queries: int = 4):
    response = llm.predict(
        query_gen_prompt, num_queries=num_queries, query=query
    )
    # assume LLM proper put each query on a newline
    queries = response.split("\n")
    queries_str = "\n".join(queries)
    print(f"Generated queries:\n{queries_str}")
    return queries

In [ ]:
queries = generate_queries("Was ist die Definition von agiler Arbeitsweise?", llm)

In [ ]:
queries

In [ ]:
# import QueryBundle
from llama_index.core import QueryBundle

# import NodeWithScore
from llama_index.core.schema import NodeWithScore

# Retrievers
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KeywordTableSimpleRetriever,
)

from typing import List

In [ ]:
from llama_index.core import Settings

documents = SimpleDirectoryReader(input_dir="./data",
    required_exts=required_exts,
    recursive=True
    ).load_data()

nodes = Settings.node_parser.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core import StorageContext

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [ ]:
from llama_index.core import SimpleKeywordTableIndex, VectorStoreIndex

vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

In [ ]:
class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both semantic search and hybrid search."""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        keyword_retriever: KeywordTableSimpleRetriever,
        mode: str = "AND",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._keyword_retriever = keyword_retriever
        
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        keyword_nodes = self._keyword_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        keyword_ids = {n.node.node_id for n in keyword_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in keyword_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(keyword_ids)
        else:
            retrieve_ids = vector_ids.union(keyword_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# define custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=6)
keyword_retriever = KeywordTableSimpleRetriever(index=keyword_index)
custom_retriever = CustomRetriever(vector_retriever, keyword_retriever)

# define response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

# vector query engine
vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)
# keyword query engine
keyword_query_engine = RetrieverQueryEngine(
    retriever=keyword_retriever,
    response_synthesizer=response_synthesizer,
)

In [ ]:
response = custom_query_engine.query("Was ist die Definition von agiler Arbeitsweise?")

In [ ]:
print(response)

In [ ]:
# in contrast, vector search will return an answer
response = vector_query_engine.retrieve("Was ist die Definition von agiler Arbeitsweise?")
response